# Tuning Result Buffer

We need to keep track of the first several best-performing hyperparameter configurations of the model. These top configurations are stored in the hyperparameter configuration buffer.

## Priority Queue

Since the tuning results should be ordered by the accuracy of the model with a certain hyperparameter configuration setting on the validation dataset, the internal implementation of the tuning result buffer should be a priority queue.

We implemented our own `PriorityQueue`.

Suppose we have the following tuning results:

In [23]:
from linguaml.families import SVCFamily

result1 = {
    "hp_config": SVCFamily.hp()(
        C=0.1,
        tol=0.01,
        gamma=0.01,
        kernel="linear",
        decision_function_shape="ovo"
    ),
    "accuracy": 0.6
}

result2 = {
    "hp_config": SVCFamily.hp()(
        C=10,
        tol=0.01,
        gamma=0.1,
        kernel="linear",
        decision_function_shape="ovo"
    ),
    "accuracy": 0.9
}

result3 = {
    "hp_config": SVCFamily.hp()(
        C=1000,
        tol=0.01,
        gamma=0.1,
        kernel="rbf",
        decision_function_shape="ovr"
    ),
    "accuracy": 0.7
}

To put them into a priority queue, we must first define the order. You may do so by wrapping each object into a custom class inheriting from `OrderedItem`, and implement the static method `get_value`.

Smaller accuracy should result in higher priority.

In [29]:
from linguaml.data.priority_queue import OrderedItem

class OrderedTuningResult(OrderedItem):
    
    def get_value(item) -> float:
        
        return -item["accuracy"]


You can then push each result into the priority queue.

In [30]:
from linguaml.data.priority_queue import PriorityQueue

# Creates an empry queue
q = PriorityQueue()

# Push items
q.push(OrderedTuningResult(result1))
q.push(OrderedTuningResult(result2))
q.push(OrderedTuningResult(result3))

The method `peek_first_n_items` returns the first several items with the highest priorities.

In [28]:
q.peek_first_n_items(3)

[(item={'hp_config': SVCConfig(C=10.0, kernel='linear', gamma=0.1, tol=0.01, decision_function_shape='ovo'), 'accuracy': 0.9}, value=-0.9),
 (item={'hp_config': SVCConfig(C=1000.0, kernel='rbf', gamma=0.1, tol=0.01, decision_function_shape='ovr'), 'accuracy': 0.7}, value=-0.7),
 (item={'hp_config': SVCConfig(C=0.1, kernel='linear', gamma=0.01, tol=0.01, decision_function_shape='ovo'), 'accuracy': 0.6}, value=-0.6)]

If you wish to push a list of items, you may use the `extend` method.

In [32]:
q = PriorityQueue()

q.extend([
    OrderedTuningResult(result1),
    OrderedTuningResult(result2),
    OrderedTuningResult(result3)
])

q.peek_first_n_items(3)

[(item={'hp_config': SVCConfig(C=10.0, kernel='linear', gamma=0.1, tol=0.01, decision_function_shape='ovo'), 'accuracy': 0.9}, value=-0.9),
 (item={'hp_config': SVCConfig(C=1000.0, kernel='rbf', gamma=0.1, tol=0.01, decision_function_shape='ovr'), 'accuracy': 0.7}, value=-0.7),
 (item={'hp_config': SVCConfig(C=0.1, kernel='linear', gamma=0.01, tol=0.01, decision_function_shape='ovo'), 'accuracy': 0.6}, value=-0.6)]